# RNA-seqの取り出し

 - 理研ID（Sxxxxxx）のリストを読み込む
 - STAR
       対応するRNA-seq（Sxxxxx.ReadsPerGene.out.tab）のcountsを集計する
 - RSEM
       Sxxxxx.genes.results, Sxxxxx.isoforms.resultsのexpected_countsを集計する

[参考]
- [STAR-RSEMによる発現量推定](https://rnakato.hatenablog.jp/entry/2018/12/28/184301)
- [Quantification using RSEM](https://ycl6.gitbook.io/rna-seq-data-analysis/rna-seq_analysis_workflow/quantification_using_rsem1)

# setup

In [1]:
import sys
import os
import glob
import pandas as pd

#RNA_seq_dir = '/interim/rnaseq-automated/GRCh38/'
RNA_seq_path = './RNA-seq/'

rikenID_file='prurigo_rikenID_bulk.csv'

# 設定ファイルを読んでrikenIDリスト作成
rikenids = pd.read_csv(rikenID_file,header=None)[0].to_list()

In [20]:
# 全RNAseqを対象にする
# 全ディレクトリ名を取得
files = os.listdir(RNA_seq_path)
#files_dir = [f for f in files if os.path.isdir(os.path.join(RNA_seq_path, f))]
files_dir = [f for f in files if os.path.isdir(os.path.join(RNA_seq_path, f))]

files_dir.sort()
rikenids = files_dir
pd.Series(rikenids).to_csv('RNAseq_sampleID_list.csv',index=False,header=False)

print(files_dir)

['S09796_L1', 'S09797', 'S09798']


In [23]:
# .ReadsPerGene.out.tab
fileName = '.ReadsPerGene.out.tab'
file_path = RNA_seq_path+'S09797'+'/'+'S09797'+fileName
if os.path.exists(file_path):
    df_tmp = pd.read_table(file_path,header=None,skiprows=4)
df_tmp.head()

,0,1,2,3
0,DDX11L1,4,5,6
1,WASH7P,1,0,1
2,MIR6859-3,0,0,0
3,MIR6859-2,0,0,0
4,MIR6859-1,0,0,0


In [24]:
print(df_tmp)

               0  1  2  3
0        DDX11L1  4  5  6
1         WASH7P  1  0  1
2      MIR6859-3  0  0  0
3      MIR6859-2  0  0  0
4      MIR6859-1  0  0  0
...          ... .. .. ..
26470      CDY1B  0  0  0
26471       CDY1  0  0  0
26472   CSPG4P1Y  0  0  0
26473  GOLGA2P3Y  0  0  0
26474  GOLGA2P2Y  9  0  0

[26475 rows x 4 columns]


# STAR (ReadsPerGene.out.tab)
ReadsPerGene.out.tabについて、geneIDを縦列、サンプル（理研ID）を横列としたRNA-seq countsのテーブルを作成する

 - ReadsPerGene.out.tab file with 4 columns which correspond to different strandedness options:
1. column 1: gene ID
2. column 2: counts for unstranded RNA-seq
3. column 3: counts for the 1st read strand aligned with RNA (htseq-count option -s yes)
4. column 4: counts for the 2nd read strand aligned with RNA (htseq-count option -s reverse)

In [21]:
# rikenIDがついたReadsPerGene.out.tabを見つけてread
fileName = '.ReadsPerGene.out.tab'

first_flag = True
join_list = []
miss_list = []



for rikenid in rikenids:
    
    file_path = RNA_seq_path+rikenid+'/'+rikenid+fileName

    if os.path.exists(file_path):
        df_tmp = pd.read_table(file_path,header=None,skiprows=4)
        # column=0 :gene ID
        df_tmp = df_tmp.rename(columns={0:'gene_id',1:rikenid}).set_index('gene_id')
        # column=1 :counts for unstranded RNA-seq
        df_tmp = df_tmp[[rikenid]]
        
        # 初回のみ
        if(first_flag):
            df = df_tmp
            first_flag = False
            join_list.append(rikenid)
        else:
            # ２回目以降
            df = df.join(df_tmp)
            join_list.append(rikenid)
    else:
        miss_list.append(rikenid)

print(file_path+' : join ID = ',join_list)
print(file_path+' : miss ID = ',miss_list)

df.to_csv('ReadsPerGene.out.tab_RNA-seq_counts.csv')
df.head()


./RNA-seq/S09798/S09798.ReadsPerGene.out.tab : join ID =  ['S09796_L1', 'S09797', 'S09798']
./RNA-seq/S09798/S09798.ReadsPerGene.out.tab : miss ID =  []


,S09796_L1,S09797,S09798
gene_id,,,
DDX11L1,1,4,7
WASH7P,0,1,1
MIR6859-3,0,0,0
MIR6859-2,0,0,0
MIR6859-1,0,0,0


# RSEM (.genes.results, .isoforms.results)
geneIDをexpected_counts縦列、サンプル（理研ID）を横列のテーブルを作成する

- .genes.results
1. gene_id
2. transcript_id(s)
3. length
4. effective_length
5. expected_count
6. TPM
7. FPKM

- .isoforms.results
1. transcript_id
2. gene_id
3. length
4. effective_length
5. expected_count
6. TPM
7. FPKM
8. IsoPct

In [22]:
# どんな内容か？
fileName = '.genes.results'
file_path = RNA_seq_path+'S09796'+'/'+'S09796'+fileName
if os.path.exists(file_path):
    df_tmp = pd.read_table(file_path,)
df_tmp.head()

,S09798
gene_id,
DDX11L1,7
WASH7P,1
MIR6859-3,0
MIR6859-2,0
MIR6859-1,0


In [107]:
# どんな内容か？
fileName = '.isoforms.results'
file_path = RNA_seq_path+'S09796'+'/'+'S09796'+fileName
if os.path.exists(file_path):
    df_tmp = pd.read_table(file_path)
df_tmp

,transcript_id,gene_id,length,effective_length,expected_count,TPM,FPKM,IsoPct
0,ENST00000373020,ENSG00000000003,2206,100.00,1.00,0.00,0.01,100.00
1,ENST00000494424,ENSG00000000003,820,611.20,0.00,0.00,0.00,0.00
2,ENST00000496771,ENSG00000000003,1025,816.20,0.00,0.00,0.00,0.00
3,ENST00000612152,ENSG00000000003,3796,3587.20,0.00,0.00,0.00,0.00
4,ENST00000614008,ENSG00000000003,900,691.20,0.00,0.00,0.00,0.00
5,ENST00000373031,ENSG00000000005,1339,1130.20,0.00,0.00,0.00,0.00
6,ENST00000485971,ENSG00000000005,542,333.26,0.00,0.00,0.00,0.00
7,ENST00000371582,ENSG00000000419,1161,952.20,16.56,0.10,0.27,0.72
8,ENST00000371584,ENSG00000000419,1073,864.20,8.54,0.06,0.15,0.41


In [9]:
# rikenIDがついた.genes.resultsを見つけてread
fileName = '.genes.results'
target_column = 'TPM'
join_list = []
miss_list = []
first_flag = True

for rikenid in rikenids:

    file_path = RNA_seq_path+rikenid+'/'+rikenid+fileName

    if os.path.exists(file_path):
        df_tmp = pd.read_table(file_path)
        # column=0 :gene_id
        df_tmp = df_tmp.rename(columns={target_column:rikenid}).set_index('gene_id')
        # column=1 :counts for unstranded RNA-seq
        df_tmp = df_tmp[[rikenid]]

        # 初回のみ
        if(first_flag):
            df = df_tmp
            first_flag = False
            join_list.append(rikenid)
        else:
            # ２回目以降
            df = df.join(df_tmp)
            join_list.append(rikenid)
    else:
        miss_list.append(rikenid)

print(fileName+' : join ID = ',join_list)
print(fileName+' : miss ID = ',miss_list)

df.to_csv('genes.results_expected_count.csv')
df.head()

.genes.results : join ID =  ['S09796_L1', 'S09798', 'S09797']
.genes.results : miss ID =  []


,S09796_L1,S09798,S09797
gene_id,,,
ENSG00000000003,40.20,40.20,40.20
ENSG00000000005,1.21,1.21,1.21
ENSG00000000419,32.34,32.34,32.34
ENSG00000000457,2.32,2.32,2.32
ENSG00000000460,18.38,18.38,18.38


In [118]:
fileName = '.isoforms.results'
join_list = []
miss_list = []
first_flag = True

for rikenid in rikenids:

    file_path = RNA_seq_path+rikenid+'/'+rikenid+fileName

    if os.path.exists(file_path):
        df_tmp = pd.read_table(file_path)
        # column=0 :gene_id
        df_tmp = df_tmp.rename(columns={'expected_count':rikenid}).set_index(['transcript_id','gene_id'])
        # column=1 :counts for unstranded RNA-seq
        df_tmp = df_tmp[[rikenid]]

        # 初回のみ
        if(first_flag):
            df = df_tmp
            first_flag = False
            join_list.append(rikenid)
        else:
            # ２回目以降
            df = df.join(df_tmp)
            join_list.append(rikenid)
    else:
        miss_list.append(rikenid)

print(fileName+' : join ID = ',join_list)
print(fileName+' : miss ID = ',miss_list)

df.to_csv('isoforms.results_expected_count.csv')
df.head()

.isoforms.results : join ID =  ['S09796', 'S09797', 'S09798']
.isoforms.results : miss ID =  ['S10869', 'S10870', 'S15431']


,,S09796,S09797,S09798
transcript_id,gene_id,,,
ENST00000373020,ENSG00000000003,1.0,1.0,1.0
ENST00000494424,ENSG00000000003,0.0,0.0,0.0
ENST00000496771,ENSG00000000003,0.0,0.0,0.0
ENST00000612152,ENSG00000000003,0.0,0.0,0.0
ENST00000614008,ENSG00000000003,0.0,0.0,0.0
